In [1]:
import pandas as pd

data = pd.read_csv('~/ctgan/data/annual_KY_WI_3class_stdDev.csv')
data.head()

,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Standard Deviation,Average Yield,Number of Deviations,Class
0,Kentucky,Lexington,8/9/2012,55V50,8/12/2013,224,7.01,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481,2.082821,5.981552,0.493777,2
1,Kentucky,Lexington,8/9/2012,Phoenix,8/12/2013,224,6.97,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481,2.082821,5.981552,0.474572,2
2,Kentucky,Lexington,8/9/2012,Evermore,8/12/2013,224,6.96,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481,2.082821,5.981552,0.469771,2
3,Kentucky,Lexington,8/9/2012,4030,8/12/2013,224,6.95,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481,2.082821,5.981552,0.464969,2
4,Kentucky,Lexington,8/9/2012,Caliber,8/12/2013,224,6.87,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481,2.082821,5.981552,0.426560,2


In [2]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

In [3]:
new_data = model.sample(1000)

In [4]:
new_data.to_csv('~/ctgan/data/synthData/rf_dt_1011_1k_stda_01.csv')
new_data.head()

,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Standard Deviation,Average Yield,Number of Deviations,Class
0,Kentucky,Lexington,41365,55V50,9/19/2016,243,7.65,130,48,3372.28,747.8,21.375482,15.451188,15.212312,0.266070,2.082821,5.981552,0.183854,1
1,Kentucky,Lexington,41365,ROADRUNNER,9/15/2017,237,6.53,347,366,3520.80,682.4,21.801494,10.088343,11.621261,0.312382,2.082821,5.981552,1.574154,2
2,Kentucky,Lexington,41365,Evermore,41890,245,5.41,325,374,4823.96,847.6,17.442411,18.878327,14.687783,0.102591,2.082821,5.981552,-1.211388,2
3,Wisconsin,Lexington,41365,AFX 064022,9/18/2014,246,2.50,130,367,4529.30,1043.2,19.740329,15.207585,25.673937,0.290318,2.082821,5.981552,-0.117908,3
4,Kentucky,Lexington,3/31/2015,55V50,9/13/2018,241,9.68,130,64,2092.11,995.0,16.868614,15.998843,27.868069,0.110619,2.082821,5.981552,-0.796159,2


In [5]:
# get aggregate data
targetDataLoc = '~/ctgan/data/annual_GA_3class_1varPerYear_stdDev.csv'
#aggDataLoc = 'data/synth1_GA_only_063022.csv'

aggDf = new_data #pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [6]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [7]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)"]

xColumnsToKeep = ["Total Radiation (MJ/m^2)","Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", 
                  "Avg Max Temp (C)","Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Class"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = targetDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = targetDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [8]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["gini"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['gini'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [(RandomForestClassifier(), rfParamGrid, "rfTup.pkl"),
              #(KNeighborsRegressor(), knnParamGrid, "knnTup.pkl"),
              #(SVR(), svrParamGrid, "svrTup.pkl"),
             #(MLPClassifier(), nnParamGrid, "nnTup.pkl")]#,
             #(LinearRegression(), linRegParamGrid, "linRegTup.pkl"),
             #(BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             (DecisionTreeClassifier(), dtParamGrid, "dtTup.pkl")]

N = 10
workingDir = 'working_dir'
numFeatures = 6 # 11

In [9]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  rfTup
Avg MAE:  948.0
Avg R squared:  -0.048
Best MAE:  940.0
Best R squared:  0.018
Parameters of the best model:  {'model__criterion': 'gini', 'model__max_depth': 5, 'model__n_estimators': 100}
Features selected by best model:  ['Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
Accuracy:  0.55
f1 score:  0.27566225165562913
mcc:  0.16037773587382184
 
test results on our test data: 
results:
MAE:  0.375
R sq:  -0.04347826086956519
accuracy score for DA/TDA:  0.625
model:  dtTup
Avg MAE:  1176.0
Avg R squared:  -0.517
Best MAE:  1200.0
Best R squared:  -0.106
Parameters of the best model:  {'model__criterion': 'gini', 'model__max_depth': 5}
Features selected by best model:  ['Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
Accuracy:  0.54
f1 score:  0.40155699403348316
mcc:  0.16944234855485543
 
test resul